In [1]:
!pip install -q --disable-pip-version-check plotly

In [2]:
import pandas as pd

class Rep:
    """
        Z-Order Interleaving Demonstration
    """

    def __init__(self, index_range=10):
        """
            Basic setup and calls
        """
        self._create_indices(index_range)
        self._mapping_setup()
        self._bin_interleave()
        self._make_coords()
        self._map_1d()
        self._create_bins()
        self._narrow_down()
        self.plot()
    
    def _create_indices(self, index_range):
        """
            Creates indices to use and the larger cross product to transform
        """
        indA = list(range(index_range))
        indA.remove(0)
        self._a_index = indA
        indB = list(range(index_range))
        indB.remove(0)
        self._b_index = indB
    
    def _mapping_setup(self):
        """
            Maps values to binary representations
        """
        tdfA = pd.DataFrame(self._a_index, columns=["indA"])
        tdfB = pd.DataFrame(self._b_index, columns=["indB"])
        tdf = tdfA.join(tdfB, on=None, how='cross')
        tdf["binA"] = tdf.indA.apply(lambda x: str(bin(x))[2:])
        tdf["binB"] = tdf.indB.apply(lambda x: str(bin(x))[2:])
        tdf.binA = tdf.binA.str.pad(width=8, side='left', fillchar='0')
        tdf.binB = tdf.binB.str.pad(width=8, side='left', fillchar='0')
        self._rep = tdf

    def _bin_interleave(self, prefix=False):
        """
            Performs the interleaving step with the binary representations
        """
        def interleave(a, b):
            int_result = []
            A = list(a)
            B = list(b)
            while (A or B):
                int_result.append(A.pop(0))
                int_result.append(B.pop(0))
            term = "".join(int_result)
            return "0b" + term if prefix else term 
        self._rep["binAB"] = self._rep.apply(lambda x: interleave(x.binA, x.binB), axis=1)

    def _make_coords(self):
        """
            Make the string version of the coordinates
        """
        self._rep["coordAB"] = self._rep.indA.astype(str).str.cat(self._rep.indB.astype(str), sep=", ")

    def _map_1d(self):
        """
            Get an integer version of the new 1d representation
        """
        self._rep["ind1D"] = self._rep.binAB.apply(lambda x: int(x, 2))

    def _narrow_down(self):
        """
            Creates narrower df with coord pair and interleaved binary
        """
        ndf = self._rep[["coordAB", "binAB", "ind1D"]].copy()
        ndf = ndf.reset_index(drop=True)
        ndf.columns = ["coordAB", "binAB", "ind1D"]
        ndf = ndf.sort_values(by=["binAB"], ignore_index=True, kind='heapsort')
        self._narrow_rep = ndf

    def _create_bins(self):
        """
            Creates binning(partitions) and then makes them identifiers
        """
        self._rep = self._rep.sort_values(by=["binAB"], ignore_index=True, kind='heapsort')
        self._rep["bin"] = pd.factorize(pd.cut(x=self._rep.ind1D, bins=10, retbins=False))[0]

    def plot(self):
        import plotly.express as px
        adf = self._rep
        adf["bin"] = adf["bin"].astype(str)
        fig = px.scatter(
            adf,
            x="indA",
            y="indB",
            color="bin",
            hover_data=['coordAB', 'binAB'],
            color_discrete_map=dict(zip(list(range(10, 0, -1)), px.colors.qualitative.T10)),
            facet_col="bin",
            facet_col_wrap=3,
            width=900,
            height=900,
            )
        fig.update_xaxes(range=[0, 10], title_text=None, showticklabels=False, showgrid=False)
        fig.update_yaxes(range=[10, 0], title_text=None, showticklabels=False, showgrid=False)
        fig.update_traces(marker_size=15)
        fig.update_xaxes(title_text="y", row=1, col=1)
        fig.update_xaxes(title_text="y", row=1, col=2)
        fig.update_xaxes(title_text="y", row=1, col=3)
        fig.update_yaxes(title_text="x", row=1, col=1)
        fig.update_yaxes(title_text="x", row=2, col=1)
        fig.update_yaxes(title_text="x", row=3, col=1)
        fig.write_html('zorder.html', auto_open=True)
        
    def get(self):
        return self._rep

    def get_narrow(self):
        return self._narrow_rep
    
    def get_markdown(self):
        return self._rep.get_narrow().to_markdown(index=False)

In [3]:
rep = Rep()
# you should see zorder.html generated in the directory
# when opening it should prompt you to "Trust HTML" after which you should see it